In [ ]:
from __future__ import annotations

In [ ]:
# Download and install English language model for spaCy
! python -m spacy download en_core_web_sm

In [ ]:
# Import the spacy, pandas, sklearn.model_selection
import spacy
nlp = spacy.load('en_core_web_sm')
import pandas as pd
from sklearn.model_selection import train_test_split

In [43]:
# Import data
df = pd.read_csv('starter/data/reviews.csv')
df.head()

,Clothing ID,Age,Title,Review Text,Positive Feedback Count,Division Name,Department Name,Class Name,Recommended IND
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,0,General,Dresses,Dresses,0
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",0,General Petite,Bottoms,Pants,1
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,6,General,Tops,Blouses,1
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",4,General,Dresses,Dresses,0
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,1,General Petite,Tops,Knits,1


In [ ]:
# Summary
df.describe()

In [ ]:
# Count nulls
df.isnull().sum()

In [52]:
# Separate features from labels
x = df.drop('Recommended IND' , axis= 1)
y = df['Recommended IND'].copy()

X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=27, shuffle=True)